In [1]:
import os
import shutil

In [23]:
# Define your directories
gt_dir = "CP/gt/"
pred_dir = "CP/predict/"
output_fp_dir = 'CP/fp'
output_fn_dir = 'CP/fn'  

In [24]:
os.makedirs(output_fp_dir, exist_ok=True)
os.makedirs(output_fn_dir, exist_ok=True)

In [25]:
def read_yolo_file(file_path, include_confidence=False):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    if include_confidence:
        objects = [line.strip().split() for line in lines]
    else:
        objects = [line.strip().split()[:] for line in lines]  # Exclude confidence score for GT
    return objects

In [26]:
def identify_fp_fn(gt_objects, pred_objects):
    # Placeholder functions for identifying FPs and FNs
    # This requires a more complex logic to compare objects
    # For simplicity, we'll just return different sets for demonstration
    fps = []  # False positives
    fns = []  # False negatives

    # Example logic to fill fps and fns lists
    gt_classes = {obj[0] for obj in gt_objects}
    pred_classes = {obj[0] for obj in pred_objects}

    # Identify false positives and false negatives
    for pred in pred_objects:
        if pred[0] not in gt_classes:
            fps.append(pred)

    for gt in gt_objects:
        if gt[0] not in pred_classes:
            fns.append(gt)

    return fps, fns

In [27]:
# Loop through the ground truth files
for gt_file in os.listdir(gt_dir):
    gt_path = os.path.join(gt_dir, gt_file)
    pred_path = os.path.join(pred_dir, gt_file)
    
    # Ensure there's a corresponding prediction file
    if os.path.exists(pred_path):
        gt_objects = read_yolo_file(gt_path,include_confidence=False)
        pred_objects = read_yolo_file(pred_path, include_confidence=True)
        
        # Identify FP and FN
        fps, fns = identify_fp_fn(gt_objects, pred_objects)

        # Save FPs and FNs in their respective directories
        if fps:  # If there are false positives
            with open(os.path.join(output_fp_dir, gt_file), 'w') as fp_file:
                for fp in fps:
                    fp_file.write(' '.join(fp) + '\n')

        if fns:  # If there are false negatives
            with open(os.path.join(output_fn_dir, gt_file), 'w') as fn_file:
                for fn in fns:
                    fn_file.write(' '.join(fn) + '\n')

print("FP and FN separation complete.")

FP and FN separation complete.


In [29]:
import os

# Define your directories
gt_dir = "CP/gt/"
pred_dir = "CP/predict/"
output_fp_dir = 'CP/fp'
output_fn_dir = 'CP/fn'
output_tp_dir = 'CP/tp'  
os.makedirs(output_fp_dir, exist_ok=True)
os.makedirs(output_fn_dir, exist_ok=True)
os.makedirs(output_tp_dir, exist_ok=True)

def read_yolo_file(file_path, include_confidence=False):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    if include_confidence:
        objects = [line.strip().split() for line in lines]
    else:
        objects = [line.strip().split()[:] for line in lines]  # Exclude confidence score for GT
    return objects

def identify_fp_fn_tp(gt_objects, pred_objects):
    fps = []  # False positives
    fns = []  # False negatives
    tps = []  # True positives

    gt_classes = {obj[0] for obj in gt_objects}
    pred_classes = {obj[0] for obj in pred_objects}

    for pred in pred_objects:
        if pred[0] in gt_classes:
            tps.append(pred)
        else:
            fps.append(pred)

    for gt in gt_objects:
        if gt[0] not in pred_classes:
            fns.append(gt)

    return fps, fns, tps

# Loop through the ground truth files
for gt_file in os.listdir(gt_dir):
    gt_path = os.path.join(gt_dir, gt_file)
    pred_path = os.path.join(pred_dir, gt_file)
    
    # Ensure there's a corresponding prediction file
    if os.path.exists(pred_path):
        gt_objects = read_yolo_file(gt_path,include_confidence=False)
        pred_objects = read_yolo_file(pred_path, include_confidence=True)
        
        # Identify FP, FN, and TP
        fps, fns, tps = identify_fp_fn_tp(gt_objects, pred_objects)

        # Save FPs, FNs, and TPs in their respective directories
        if fps:
            with open(os.path.join(output_fp_dir, gt_file), 'w') as fp_file:
                for fp in fps:
                    fp_file.write(' '.join(fp) + '\n')

        if fns:
            with open(os.path.join(output_fn_dir, gt_file), 'w') as fn_file:
                for fn in fns:
                    fn_file.write(' '.join(fn) + '\n')

        if tps:
            with open(os.path.join(output_tp_dir, gt_file), 'w') as tp_file:
                for tp in tps:
                    tp_file.write(' '.join(tp) + '\n')

print("FP, FN, and TP separation complete.")


FP, FN, and TP separation complete.


In [31]:
import os
from collections import defaultdict

def count_objects_per_class(folder_path):
    object_count = defaultdict(int)

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    class_id = int(line.split()[0])
                    object_count[class_id] += 1

    return object_count

folder_path = "CP/tp"
result = count_objects_per_class(folder_path)
print(result)

defaultdict(<class 'int'>, {1: 492, 2: 53, 3: 170, 0: 6911})


# move files
## Schisto

In [42]:
import os
import shutil

# Function to move files with object ID 3 to a separate folder
def move_files_with_object_id_3(prediction_folder, output_prediction_folder, ground_truth_folder, output_ground_truth_folder):
    # Create output folders if they don't exist
    os.makedirs(output_prediction_folder, exist_ok=True)
    os.makedirs(output_ground_truth_folder, exist_ok=True)
    
    # List all files in the prediction folder
    prediction_files = os.listdir(prediction_folder)
    
    for prediction_file in prediction_files:
        # Read the prediction file
        with open(os.path.join(prediction_folder, prediction_file), 'r') as f:
            lines = f.readlines()
        
        # Check if any line contains object ID 3
        contains_object_id_3 = any(line.startswith('2 ') for line in lines)
        
        if contains_object_id_3:
            # Move the prediction file to the output prediction folder
            shutil.move(os.path.join(prediction_folder, prediction_file), os.path.join(output_prediction_folder, prediction_file))
            
            # Find and copy the corresponding ground truth file
            ground_truth_file = prediction_file  # Assuming ground truth file has the same name
            shutil.copy(os.path.join(ground_truth_folder, ground_truth_file), os.path.join(output_ground_truth_folder, ground_truth_file))

# Example usage
prediction_folder = "CP/tp"
output_prediction_folder = "CP/Hookworm_TP"
ground_truth_folder = "CP/gt/"
output_ground_truth_folder = "CP/Hookworm_GT"

move_files_with_object_id_3(prediction_folder, output_prediction_folder, ground_truth_folder, output_ground_truth_folder)


In [43]:
import os
import shutil

# Function to move files with object ID 3 to a separate folder
def move_files_with_object_id_3(prediction_folder, output_prediction_folder, ground_truth_folder, output_ground_truth_folder):
    # Create output folders if they don't exist
    os.makedirs(output_prediction_folder, exist_ok=True)
    os.makedirs(output_ground_truth_folder, exist_ok=True)
    
    # List all files in the prediction folder
    prediction_files = os.listdir(prediction_folder)
    
    for prediction_file in prediction_files:
        # Read the prediction file
        with open(os.path.join(prediction_folder, prediction_file), 'r') as f:
            lines = f.readlines()
        
        # Check if any line contains object ID 3
        contains_object_id_3 = any(line.startswith('3 ') for line in lines)
        
        if contains_object_id_3:
            # Move the prediction file to the output prediction folder
            shutil.move(os.path.join(prediction_folder, prediction_file), os.path.join(output_prediction_folder, prediction_file))
            
            # Find and copy the corresponding ground truth file
            ground_truth_file = prediction_file  # Assuming ground truth file has the same name
            shutil.copy(os.path.join(ground_truth_folder, ground_truth_file), os.path.join(output_ground_truth_folder, ground_truth_file))

# Example usage
prediction_folder = "CP/tp"
output_prediction_folder = "CP/Schisto_TP"
ground_truth_folder = "CP/gt/"
output_ground_truth_folder = "CP/Schisto_GT"

move_files_with_object_id_3(prediction_folder, output_prediction_folder, ground_truth_folder, output_ground_truth_folder)


In [46]:
import os
import shutil

# Function to move files with object ID 3 to a separate folder
def move_files_with_object_id_3(prediction_folder, output_prediction_folder, ground_truth_folder, output_ground_truth_folder):
    # Create output folders if they don't exist
    os.makedirs(output_prediction_folder, exist_ok=True)
    os.makedirs(output_ground_truth_folder, exist_ok=True)
    
    # List all files in the prediction folder
    prediction_files = os.listdir(prediction_folder)
    
    for prediction_file in prediction_files:
        # Read the prediction file
        with open(os.path.join(prediction_folder, prediction_file), 'r') as f:
            lines = f.readlines()
        
        # Check if any line contains object ID 3
        contains_object_id_3 = any(line.startswith('1 ') for line in lines)
        
        if contains_object_id_3:
            # Move the prediction file to the output prediction folder
            shutil.move(os.path.join(prediction_folder, prediction_file), os.path.join(output_prediction_folder, prediction_file))
            
            # Find and copy the corresponding ground truth file
            ground_truth_file = prediction_file  # Assuming ground truth file has the same name
            shutil.copy(os.path.join(ground_truth_folder, ground_truth_file), os.path.join(output_ground_truth_folder, ground_truth_file))

# Example usage
prediction_folder = "CP/tp"
output_prediction_folder = "CP/Trichuris_TP"
ground_truth_folder = "CP/gt/"
output_ground_truth_folder = "CP/Trichuris_GT"

move_files_with_object_id_3(prediction_folder, output_prediction_folder, ground_truth_folder, output_ground_truth_folder)


In [60]:
import os
import shutil

# Function to copy annotation files until the number of object with id 1 reaches 50
def copy_files_until_object_id_1_reaches_50(annotation_folder, gt_folder, output_folder, output_gt_folder):
    # Create output folders if they don't exist
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(output_gt_folder, exist_ok=True)
    
    # List all annotation files
    annotation_files = os.listdir(annotation_folder)
    
    count_object_id_1 = 0
    for annotation_file in annotation_files:
        # Read the annotation file
        with open(os.path.join(annotation_folder, annotation_file), 'r') as f:
            annotation_lines = f.readlines()
        
        # Read the corresponding ground truth file
        gt_file = annotation_file  # Assuming ground truth file has the same name
        with open(os.path.join(gt_folder, gt_file), 'r') as f:
            gt_lines = f.readlines()
        
        # Count the occurrences of object ID 1
        count_object_id_1 += sum(1 for line in annotation_lines if line.startswith('3 '))
        
        # Copy the annotation file to the output folder
        shutil.copy(os.path.join(annotation_folder, annotation_file), os.path.join(output_folder, annotation_file))
        
        # Copy the corresponding ground truth file to the output ground truth folder
        shutil.copy(os.path.join(gt_folder, gt_file), os.path.join(output_gt_folder, gt_file))
        
        # If count_object_id_1 reaches 50, break the loop
        if count_object_id_1 >= 73:
            break

# Example usage
annotation_folder = "CP/Schisto_TP/"
gt_folder = "CP/Schisto_GT/"
output_folder = "CP/Schisto_TP_50/"
output_gt_folder = "CP/Schisto_GT_50/"

copy_files_until_object_id_1_reaches_50(annotation_folder, gt_folder, output_folder, output_gt_folder)

In [27]:
import os
from collections import defaultdict

def count_objects_per_class(folder_path):
    object_count = defaultdict(int)

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    class_id = int(line.split()[0])
                    object_count[class_id] += 1

    return object_count

folder_path = "CP/CP_FINAL_GT_aligned//"
result = count_objects_per_class(folder_path)
print(result)

defaultdict(<class 'int'>, {1: 49, 2: 47, 3: 52, 0: 172})


# It doesn't work based on on my expectations 

In [14]:
import os

# Function to parse annotations from a YOLO format file
def parse_annotations_yolo(annotation_file):
    annotations = []
    with open(annotation_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            obj_id = int(parts[0])
            values = list(map(float, parts[1:]))  # Convert all values to float
            annotations.append((obj_id, *values))  # Unpack values
    return annotations

# Function to calculate intersection over union (IoU) between two bounding boxes
def calculate_iou(box1, box2):
    x_left = max(box1[1] - box1[3] / 2, box2[1] - box2[3] / 2)
    y_top = max(box1[2] - box1[4] / 2, box2[2] - box2[4] / 2)
    x_right = min(box1[1] + box1[3] / 2, box2[1] + box2[3] / 2)
    y_bottom = min(box1[2] + box1[4] / 2, box2[2] + box2[4] / 2)

    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)
    box1_area = box1[3] * box1[4]
    box2_area = box2[3] * box2[4]
    union_area = box1_area + box2_area - intersection_area

    iou = intersection_area / union_area if union_area > 0 else 0

    return iou

# Function to compare and keep aligned objects between GT and prediction annotation files
def keep_aligned_objects(gt_folder, pred_folder, output_gt_folder, output_pred_folder, alignment_threshold=0.5):
    os.makedirs(output_gt_folder, exist_ok=True)
    os.makedirs(output_pred_folder, exist_ok=True)
    
    gt_files = os.listdir(gt_folder)
    pred_files = os.listdir(pred_folder)
    
    for gt_file in gt_files:
        gt_annotations = parse_annotations_yolo(os.path.join(gt_folder, gt_file))
        pred_file = gt_file  # Assuming the prediction file has the same name as the GT file
        if pred_file in pred_files:
            pred_annotations = parse_annotations_yolo(os.path.join(pred_folder, pred_file))
            aligned_gt_annotations = []
            aligned_pred_annotations = []
            for gt_annotation in gt_annotations:
                for pred_annotation in pred_annotations:
                    iou = calculate_iou(gt_annotation, pred_annotation)
                    if iou >= alignment_threshold:
                        aligned_gt_annotations.append(gt_annotation)
                        aligned_pred_annotations.append(pred_annotation)
                        break  # If one aligned object is found, no need to check further
            # Write aligned annotations to new GT and prediction files
            with open(os.path.join(output_gt_folder, gt_file), 'w') as f:
                for annotation in aligned_gt_annotations:
                    f.write(' '.join(str(x) for x in annotation) + '\n')
            with open(os.path.join(output_pred_folder, pred_file), 'w') as f:
                for annotation in aligned_pred_annotations:
                    f.write(' '.join(str(x) for x in annotation) + '\n')

# Example usage
gt_folder = "CP/CP_FINAL_GT/"
pred_folder = "CP/CP_FINAL_TP/"
output_gt_folder = "CP/CP_FINAL_GT_aligned/"
output_pred_folder = "CP/CP_FINAL_TP_aligned/"

keep_aligned_objects(gt_folder, pred_folder, output_gt_folder, output_pred_folder)

In [18]:
import os

def truncate_prediction_files(prediction_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(prediction_folder):
        with open(os.path.join(prediction_folder, filename), 'r') as file:
            lines = file.readlines()

        # Truncate each line after the 5th value (object height)
        truncated_lines = [' '.join(line.split()[:5]) + '\n' for line in lines]

        # Write truncated lines to new file
        with open(os.path.join(output_folder, filename), 'w') as file:
            file.writelines(truncated_lines)

# Example usage
prediction_folder = "CP/CP_FINAL_TP/"
output_folder = "CP/CP_FINAL_TP_truncated/"

truncate_prediction_files(prediction_folder, output_folder)


In [20]:
import os
import shutil

def copy_images_based_on_annotations(image_folder, annotation_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    # Get list of annotation filenames without extensions
    annotation_filenames = [os.path.splitext(filename)[0] for filename in os.listdir(annotation_folder)]

    # Iterate through images and copy them if their corresponding annotation exists
    for filename in os.listdir(image_folder):
        image_name_without_extension = os.path.splitext(filename)[0]
        if image_name_without_extension in annotation_filenames:
            shutil.copy(os.path.join(image_folder, filename), os.path.join(output_folder, filename))

# Example usage
image_folder = "CP/images/"
annotation_folder = "CP/CP_FINAL_GT/"
output_folder = "CP/images_with_annotations_aligned/"

copy_images_based_on_annotations(image_folder, annotation_folder, output_folder)


In [1]:
import os
import csv

# Function to extract annotations from annotation files and store them in a CSV file
def create_annotation_csv(annotation_folder, output_csv_file):
    # List all annotation files
    annotation_files = os.listdir(annotation_folder)
    
    with open(output_csv_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write header
        csv_writer.writerow(['File Name', 'Object ID', 'X Center', 'Y Center', 'Width', 'Height'])
        
        # Process each annotation file
        for annotation_file in annotation_files:
            # Read the annotation file
            with open(os.path.join(annotation_folder, annotation_file), 'r') as f:
                annotation_lines = f.readlines()
            
            # Extract annotations for each object
            for line in annotation_lines:
                parts = line.strip().split()
                obj_id = parts[0]
                x_center = parts[1]
                y_center = parts[2]
                obj_width = parts[3]
                obj_height = parts[4]
                # conf_score = parts[5]
                
                # Write object details to CSV file
                csv_writer.writerow([annotation_file, obj_id, x_center, y_center, obj_width, obj_height])

# Example usage
annotation_folder = "CP/CP_FINAL_GT/"
output_csv_file = "CP/GT.csv"

create_annotation_csv(annotation_folder, output_csv_file)


In [ ]:
import os
import csv

# Function to extract annotations from annotation files and store them in a CSV file
def create_annotation_csv(annotation_folder, output_csv_file):
    # List all annotation files
    annotation_files = os.listdir(annotation_folder)
    
    with open(output_csv_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write header
        csv_writer.writerow(['File Name', 'Object ID', 'X Center', 'Y Center', 'Width', 'Height'])
        
        # Process each annotation file
        for annotation_file in annotation_files:
            # Read the annotation file
            with open(os.path.join(annotation_folder, annotation_file), 'r') as f:
                annotation_lines = f.readlines()
            
            # Extract annotations for each object
            for line in annotation_lines:
                parts = line.strip().split()
                obj_id = parts[0]
                x_center = parts[1]
                y_center = parts[2]
                obj_width = parts[3]
                obj_height = parts[4]
                # conf_score = parts[5]
                
                # Write object details to CSV file
                csv_writer.writerow([annotation_file, obj_id, x_center, y_center, obj_width, obj_height])

# Example usage
annotation_folder = "CP/CP_FINAL_GT/"
output_csv_file = "CP/GT.csv"

create_annotation_csv(annotation_folder, output_csv_file)


In [2]:
import os
import csv

# Function to extract annotations from annotation files and store them in a CSV file
def create_annotation_csv(annotation_folder, output_csv_file):
    # List all annotation files
    annotation_files = os.listdir(annotation_folder)
    
    with open(output_csv_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write header
        csv_writer.writerow(['File Name', 'Object ID', 'X Center', 'Y Center', 'Width', 'Height', 'Confidence Score', 'Object Score', 'Ascaris Confidence Score', 'Trichuris Confidence Score', 'Hookworm Confidence Score', 'Schistosoma Confidence Score'])
        
        # Process each annotation file
        for annotation_file in annotation_files:
            # Read the annotation file
            with open(os.path.join(annotation_folder, annotation_file), 'r') as f:
                annotation_lines = f.readlines()
            
            # Extract annotations for each object
            for line in annotation_lines:
                parts = line.strip().split()
                obj_id = parts[0]
                x_center = parts[1]
                y_center = parts[2]
                obj_width = parts[3]
                obj_height = parts[4]
                conf_score = parts[5]
                obj_score = parts[6]
                ascaris_conf_score = parts[7]
                trichuris_conf_score = parts[8]
                hookworm_conf_score = parts[9]
                schistosoma_conf_score = parts[10]
                
                # Write object details to CSV file
                csv_writer.writerow([annotation_file, obj_id, x_center, y_center, obj_width, obj_height, conf_score, obj_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score])

# Example usage
annotation_folder = "CP/CP_FINAL_TP/"
output_csv_file = "CP/TP.csv"

create_annotation_csv(annotation_folder, output_csv_file)

In [22]:
import os

# Define your directories
gt_dir = "CP/CP_FINAL_GT - Copy/"
pred_dir = "CP/CP_FINAL_TP - Copy/"
output_tp_dir = "CP/CP_FINAL_TP_only"
os.makedirs(output_tp_dir, exist_ok=True)

def read_yolo_file(file_path, include_confidence=False):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    if include_confidence:
        objects = [line.strip().split() for line in lines]
    else:
        objects = [line.strip().split()[:] for line in lines]  # Exclude confidence score for GT
    return objects

def identify_tp(gt_objects, pred_objects):
    tps = []  # True positives

    gt_classes = {obj[0] for obj in gt_objects}
    pred_classes = {obj[0] for obj in pred_objects}

    for pred in pred_objects:
        if pred[0] in gt_classes:
            tps.append(pred)

    return tps

# Loop through the ground truth files
for gt_file in os.listdir(gt_dir):
    gt_path = os.path.join(gt_dir, gt_file)
    pred_path = os.path.join(pred_dir, gt_file)
    
    # Ensure there's a corresponding prediction file
    if os.path.exists(pred_path):
        gt_objects = read_yolo_file(gt_path,include_confidence=False)
        pred_objects = read_yolo_file(pred_path, include_confidence=True)
        
        # Identify TP
        tps = identify_tp(gt_objects, pred_objects)

        # Save TPs in their respective directories
        if tps:
            with open(os.path.join(output_tp_dir, gt_file), 'w') as tp_file:
                for tp in tps:
                    tp_file.write(' '.join(tp) + '\n')

print("TP extraction complete.")


TP extraction complete.


In [3]:
import os
import csv
from collections import defaultdict
from itertools import product
from PIL import Image

# Function to calculate IoU
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calculate coordinates of intersection rectangle
    x_left = max(x1 - w1 / 2, x2 - w2 / 2)
    y_top = max(y1 - h1 / 2, y2 - h2 / 2)
    x_right = min(x1 + w1 / 2, x2 + w2 / 2)
    y_bottom = min(y1 + h1 / 2, y2 + h2 / 2)

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Calculate intersection area
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate area of each bounding box
    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate IoU
    iou = intersection_area / (box1_area + box2_area - intersection_area)
    return iou

# Folder paths
images_folder = "CP/images_with_annotations_aligned/"
gt_folder = "CP/CP_FINAL_GT_aligned/"
prediction_folder = "CP/CP_FINAL_TP_aligned/"

# Output CSV file
output_csv = "CP/output.csv"

# Dictionary to store annotations
annotations = defaultdict(list)

# Iterate through images folder
for filename in os.listdir(images_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Get image dimensions
        img = Image.open(os.path.join(images_folder, filename))
        img_width, img_height = img.size

        # Get corresponding ground truth and prediction files
        gt_file = os.path.join(gt_folder, filename.replace(".jpg", ".txt").replace(".png", ".txt"))
        pred_file = os.path.join(prediction_folder, filename.replace(".jpg", ".txt").replace(".png", ".txt"))

        # Read GT annotations
        with open(gt_file, "r") as f:
            gt_annotations = [list(map(float, line.strip().split())) for line in f]

        # Read prediction annotations
        with open(pred_file, "r") as f:
            pred_annotations = [list(map(float, line.strip().split())) for line in f]

        # Align GT and prediction annotations based on IoU
        for gt_obj in gt_annotations:
            gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height = gt_obj
            matched_pred = None
            max_iou = 0

            for pred_obj in pred_annotations:
                iou = calculate_iou((gt_x_center, gt_y_center, gt_obj_width, gt_obj_height), pred_obj[1:5])
                if iou > max_iou:
                    max_iou = iou
                    matched_pred = pred_obj

            if max_iou >= 0.5:
                pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score = matched_pred
                annotations[filename].append((filename, img_width, img_height, gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height, pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score))

# Write annotations to CSV
with open(output_csv, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["filename", "image_width", "image_height", "gt_obj_id", "gt_x_center", "gt_y_center", "gt_obj_width", "gt_obj_height", "pred_obj_id", "pred_x_center", "pred_y_center", "pred_obj_width", "pred_obj_height", "conf_score", "objectness_score", "ascaris_conf_score", "trichuris_conf_score", "hookworm_conf_score", "schistosoma_conf_score"])
    for filename, ann_list in annotations.items():
        for ann in ann_list:
            csv_writer.writerow(ann)


In [6]:
import os
import csv
from collections import defaultdict
from itertools import product
from PIL import Image

# Function to calculate IoU
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calculate coordinates of intersection rectangle
    x_left = max(x1 - w1 / 2, x2 - w2 / 2)
    y_top = max(y1 - h1 / 2, y2 - h2 / 2)
    x_right = min(x1 + w1 / 2, x2 + w2 / 2)
    y_bottom = min(y1 + h1 / 2, y2 + h2 / 2)

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Calculate intersection area
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate area of each bounding box
    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate IoU
    iou = intersection_area / (box1_area + box2_area - intersection_area)
    return iou

# Folder paths
images_folder = "CP/GrTru/images/"
gt_folder = "CP/GrTru/labels/"
prediction_folder = "runs/detect/conformal_prediction_25/labels/"

# Output CSV file
output_csv = "CP/conf_25.csv"

# Dictionary to store annotations
annotations = defaultdict(list)

# Set to keep track of matched predictions
matched_predictions = set()

# Iterate through images folder
for filename in os.listdir(images_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Get image dimensions
        img = Image.open(os.path.join(images_folder, filename))
        img_width, img_height = img.size

        # Get corresponding ground truth and prediction files
        gt_file = os.path.join(gt_folder, filename.replace(".jpg", ".txt").replace(".png", ".txt"))
        pred_file = os.path.join(prediction_folder, filename.replace(".jpg", ".txt").replace(".png", ".txt"))

        # Read GT annotations
        with open(gt_file, "r") as f:
            gt_annotations = [list(map(float, line.strip().split())) for line in f]

        # Read prediction annotations
        with open(pred_file, "r") as f:
            pred_annotations = [list(map(float, line.strip().split())) for line in f]

        # Align GT and prediction annotations based on IoU
        for gt_obj in gt_annotations:
            gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height = gt_obj
            matched_pred = None
            max_iou = 0

            for pred_obj in pred_annotations:
                iou = calculate_iou((gt_x_center, gt_y_center, gt_obj_width, gt_obj_height), pred_obj[1:5])
                if iou > max_iou:
                    max_iou = iou
                    matched_pred = pred_obj

            if max_iou >= 0.5:
                pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score = matched_pred
                annotations[filename].append((filename, img_width, img_height, gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height, pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score))
                matched_predictions.add(tuple(matched_pred))
            else:
                # If IoU is less than 0.5, it's a false negative
                annotations[filename].append((filename, img_width, img_height, gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height, "", "", "", "", "", "", "", "", "", "", ""))
        
        # Look for false positive predictions
        for pred_obj in pred_annotations:
            if tuple(pred_obj) not in matched_predictions:
                pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score = pred_obj
                annotations[filename].append((filename, img_width, img_height, "", "", "", "", "", pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score))

# Write annotations to CSV
with open(output_csv, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["filename", "image_width", "image_height", "gt_obj_id", "gt_x_center", "gt_y_center", "gt_obj_width", "gt_obj_height", "pred_obj_id", "pred_x_center", "pred_y_center", "pred_obj_width", "pred_obj_height", "conf_score", "objectness_score", "ascaris_conf_score", "trichuris_conf_score", "hookworm_conf_score", "schistosoma_conf_score"])
    for filename, ann_list in annotations.items():
        for ann in ann_list:
            csv_writer.writerow(ann)


In [7]:
import os
import csv
from collections import defaultdict
from itertools import product
from PIL import Image

# Function to calculate IoU
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calculate coordinates of intersection rectangle
    x_left = max(x1 - w1 / 2, x2 - w2 / 2)
    y_top = max(y1 - h1 / 2, y2 - h2 / 2)
    x_right = min(x1 + w1 / 2, x2 + w2 / 2)
    y_bottom = min(y1 + h1 / 2, y2 + h2 / 2)

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Calculate intersection area
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate area of each bounding box
    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate IoU
    iou = intersection_area / (box1_area + box2_area - intersection_area)
    return iou

# Folder paths
images_folder = "CP/GrTru/images/"
gt_folder = "CP/GrTru/labels/"
prediction_folder = "runs/detect/conformal_prediction_25/labels/"

# Output CSV file
output_csv = "CP/25.csv"

# Dictionary to store annotations
annotations = defaultdict(list)

# Set to keep track of matched predictions
matched_predictions = set()

# Iterate through images folder
for filename in os.listdir(images_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Get image dimensions
        img = Image.open(os.path.join(images_folder, filename))
        img_width, img_height = img.size

        # Get corresponding ground truth and prediction files
        gt_file = os.path.join(gt_folder, filename.replace(".jpg", ".txt").replace(".png", ".txt"))
        pred_file = os.path.join(prediction_folder, filename.replace(".jpg", ".txt").replace(".png", ".txt"))

        # Read GT annotations
        with open(gt_file, "r") as f:
            gt_annotations = [list(map(float, line.strip().split())) for line in f]

        # Read prediction annotations
        with open(pred_file, "r") as f:
            pred_annotations = [list(map(float, line.strip().split())) for line in f]

        # Align GT and prediction annotations based on IoU
        for gt_obj in gt_annotations:
            gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height = gt_obj
            matched_pred = None
            max_iou = 0

            for pred_obj in pred_annotations:
                iou = calculate_iou((gt_x_center, gt_y_center, gt_obj_width, gt_obj_height), pred_obj[1:5])
                if iou > max_iou:
                    max_iou = iou
                    matched_pred = pred_obj

            if max_iou >= 0.5:
                pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score = matched_pred
                annotations[filename].append((filename, img_width, img_height, gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height, pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score))
                matched_predictions.add(tuple(matched_pred))
            else:
                # If IoU is less than 0.5, it's a false negative
                annotations[filename].append((filename, img_width, img_height, gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height, "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", ""))
        
        # Look for false positive predictions
        for pred_obj in pred_annotations:
            if tuple(pred_obj) not in matched_predictions:
                pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score = pred_obj
                annotations[filename].append((filename, img_width, img_height, "-", "-", "-", "-", "-", pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score))

# Write annotations to CSV
with open(output_csv, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["filename", "image_width", "image_height", "gt_obj_id", "gt_x_center", "gt_y_center", "gt_obj_width", "gt_obj_height", "pred_obj_id", "pred_x_center", "pred_y_center", "pred_obj_width", "pred_obj_height", "conf_score", "objectness_score", "ascaris_conf_score", "trichuris_conf_score", "hookworm_conf_score", "schistosoma_conf_score"])
    for filename, ann_list in annotations.items():
        for ann in ann_list:
            csv_writer.writerow(ann)


In [6]:
import os
import csv
from collections import defaultdict
from itertools import product
from PIL import Image

# Function to calculate IoU
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calculate coordinates of intersection rectangle
    x_left = max(x1 - w1 / 2, x2 - w2 / 2)
    y_top = max(y1 - h1 / 2, y2 - h2 / 2)
    x_right = min(x1 + w1 / 2, x2 + w2 / 2)
    y_bottom = min(y1 + h1 / 2, y2 + h2 / 2)

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Calculate intersection area
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate area of each bounding box
    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate IoU
    iou = intersection_area / (box1_area + box2_area - intersection_area)
    return iou

# Folder paths
images_folder = "CP/GrTru/test/images/"
gt_folder = "CP/GrTru/test/labels/"
prediction_folder = "runs/detect/test_25/labels/"

# Output CSV file
output_csv = "CP/test_conf_score_25.csv"

# Dictionary to store annotations
annotations = defaultdict(list)

# Set to keep track of matched predictions
matched_predictions = set()

# Iterate through images folder
for filename in os.listdir(images_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Get image dimensions
        img = Image.open(os.path.join(images_folder, filename))
        img_width, img_height = img.size

        # Get corresponding ground truth and prediction files
        gt_file = os.path.join(gt_folder, filename.replace(".jpg", ".txt").replace(".png", ".txt"))
        pred_file = os.path.join(prediction_folder, filename.replace(".jpg", ".txt").replace(".png", ".txt"))

        # Read GT annotations
        with open(gt_file, "r") as f:
            gt_annotations = [list(map(float, line.strip().split())) for line in f]

        # Read prediction annotations
        with open(pred_file, "r") as f:
            pred_annotations = [list(map(float, line.strip().split())) for line in f]

        # Align GT and prediction annotations based on IoU
        for gt_obj in gt_annotations:
            gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height = gt_obj
            matched_pred = None
            max_iou = 0

            for pred_obj in pred_annotations:
                iou = calculate_iou((gt_x_center, gt_y_center, gt_obj_width, gt_obj_height), pred_obj[1:5])
                if iou > max_iou:
                    max_iou = iou
                    matched_pred = pred_obj

            if max_iou >= 0.5:
                pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score = matched_pred
                annotations[filename].append((filename, img_width, img_height, gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height, pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score))
                matched_predictions.add(tuple(matched_pred))
            else:
                # If IoU is less than 0.5, it's a false negative
                annotations[filename].append((filename, img_width, img_height, gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height, "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", ""))
        
        # Look for false positive predictions
        for pred_obj in pred_annotations:
            if tuple(pred_obj) not in matched_predictions:
                pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score = pred_obj
                annotations[filename].append((filename, img_width, img_height, "-", "-", "-", "-", "-", pred_obj_id, pred_x_center, pred_y_center, pred_obj_width, pred_obj_height, conf_score, objectness_score, ascaris_conf_score, trichuris_conf_score, hookworm_conf_score, schistosoma_conf_score))

# Write annotations to CSV
with open(output_csv, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["filename", "image_width", "image_height", "gt_obj_id", "gt_x_center", "gt_y_center", "gt_obj_width", "gt_obj_height", "pred_obj_id", "pred_x_center", "pred_y_center", "pred_obj_width", "pred_obj_height", "conf_score", "objectness_score", "ascaris_conf_score", "trichuris_conf_score", "hookworm_conf_score", "schistosoma_conf_score"])
    for filename, ann_list in annotations.items():
        for ann in ann_list:
            csv_writer.writerow(ann)


FileNotFoundError: [Errno 2] No such file or directory: 'CP/old_dataset/test_old_u/test_0.txt'